In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import warnings
from functools import reduce

# jtplot.style(theme=’monokai’, context=’notebook’, ticks=True, grid=False)
# jt -t oceans16 -f fira -fs 11 -nf ptsans -nfs 13 -N -kl -cursw 5 -cursc r -cellw 85% -T

## Dataset Cleaning & Preprocessing

### 1. O* NET 

In [24]:
variables = {
    'Social_Perceptiveness.csv': 'SP',
    'Persuasion.csv': 'P',
    'Finger_Dexterity.csv': 'FD',
    'Originality.csv': 'O',
    'Assisting_and_Caring_for_Others.csv': 'AC',
    'Manual_Dexterity.csv': 'MD',
    'Cramped_Work_Space_Awkward_Positions.csv': 'CW',
    'Negotiation.csv': 'N',
    'Fine_Arts.csv': 'FA'
}

onetDB = pd.DataFrame()
for fileLoc, heading in variables.items():
    data = pd.read_csv("data/raw/" + fileLoc)
    if "Importance" in data.columns:
        data.rename(columns = {'Importance' : heading}, inplace=True)
        del data['Level']
    else: data.rename(columns={'Context': heading}, inplace=True)
        
    if onetDB.empty: onetDB = data.copy()
    else: onetDB = pd.merge(onetDB, data, on=['Code', 'Occupation'])
onetDB["Code"] = onetDB["Code"].apply(lambda str: str[:-3])
onetDB["FA"] = onetDB["FA"].apply(lambda value: value if value!='Not available' else '0')
onetDB = onetDB[['Code', 'Occupation','SP' ,'P', 'FD', 'O', 'AC', 'MD', 'CW', 'N', 'FA']]
onetDB = onetDB.astype({'Code':'str','Occupation':'str', 'FA':'int64'})

print("onetDB Dimensions =>", onetDB.shape)
onetDB.head()

onetDB Dimensions => (968, 11)


,Code,Occupation,SP,P,FD,O,AC,MD,CW,N,FA
0,19-3031,Counseling Psychologists,94,72,19,66,91,0,0,69,5
1,21-1014,Mental Health Counselors,94,66,28,60,94,0,3,56,15
2,19-3031,Clinical Psychologists,94,60,22,56,99,0,5,50,5
3,29-1066,Psychiatrists,88,63,35,56,99,0,6,56,5
4,21-1013,Marriage and Family Therapists,81,72,38,66,94,0,1,69,5


### 2. SOC

In [25]:
socDB = pd.read_csv("data/raw/all_data_M_2018.csv", usecols=["own_code", "occ_code", "occ_title",
                                                          "tot_emp", "loc_quotient", "pct_total", "h_mean", "a_mean", "h_median", "a_median"])
socDB = socDB.drop(['loc_quotient','pct_total','h_median','a_median'], axis=1)
socDB.rename(columns={'own_code':'Industry Code','occ_code':'Code','tot_emp':'Total Emp', 
                      'h_mean': 'Hourly Income', 'a_mean':'Annual Income','occ_title':'Occupation'}, inplace=True)
socDB = socDB.drop_duplicates(subset=['Occupation'])
delComma = lambda no: no.replace(',','') if no != '*' else None
socDB['Total Emp'] = socDB['Total Emp'].apply(delComma)
socDB['Annual Income'] = socDB['Annual Income'].apply(delComma)
socDB['Hourly Income'] = socDB['Hourly Income'].apply(delComma)
socDB = socDB.astype({'Industry Code':'str', 'Total Emp':'int64', 'Hourly Income':'float64', 'Annual Income': 'float64' })
socDB = socDB[['Code', 'Occupation','Industry Code' ,'Total Emp', 'Hourly Income','Annual Income']]

print("socDB Dimensions =>", socDB.shape)
socDB.head()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


socDB Dimensions => (1103, 6)


,Code,Occupation,Industry Code,Total Emp,Hourly Income,Annual Income
0,00-0000,All Occupations,1235,144733270,24.98,51960.0
1,11-0000,Management Occupations,1235,7616650,58.44,121560.0
2,11-1000,Top Executives,1235,2535640,61.66,128240.0
3,11-1010,Chief Executives,1235,195530,96.22,200140.0
5,11-1020,General and Operations Managers,1235,2289770,59.56,123880.0


In [26]:
socDB = pd.read_csv("data/raw/all_data_M_2017.csv", usecols=["own_code", "occ_code", "occ_title",
                                                          "tot_emp", "loc_quotient", "pct_total", "h_mean", "h_median"])
socDB = socDB.drop(['loc_quotient','pct_total','h_median'], axis=1)
socDB.rename(columns={'own_code':'Industry Code','occ_code':'Code','tot_emp':'Total Emp', 
                      'h_mean': 'Hourly Income', 'a_mean':'Annual Income','occ_title':'Occupation'}, inplace=True)
socDB = socDB.drop_duplicates(subset=['Occupation'])
delComma = lambda no: no.replace(',','') if no != '*' else None
socDB['Total Emp'] = socDB['Total Emp'].apply(delComma)
socDB['Hourly Income'] = socDB['Hourly Income'].apply(delComma)
socDB = socDB.astype({'Industry Code':'str', 'Total Emp':'int64', 'Hourly Income':'float64' })
socDB = socDB[['Code', 'Occupation','Industry Code' ,'Total Emp', 'Hourly Income']]

print("socDB Dimensions =>", socDB.shape)
socDB.head()

socDB Dimensions => (1106, 5)


,Code,Occupation,Industry Code,Total Emp,Hourly Income
0,00-0000,All Occupations,1235,142549250,24.34
1,11-0000,Management Occupations,1235,7280330,57.65
2,11-1000,Top Executives,1235,2473740,61.55
3,11-1010,Chief Executives,1235,210160,94.25
5,11-1020,General and Operations Managers,1235,2212200,59.35


In [13]:
socDB.to_csv('socDB.csv', index=False)

### 3. Oxford

In [27]:
oxfordDB = pd.read_csv("data/raw/oxford_data.csv")
oxfordDB.rename(columns={'code': "Code", 'occupation': 'Occupation', 'rank':'Rank', 'probability':'Probability','label':'Label'}, inplace=True)
oxfordDB = oxfordDB[['Code', 'Occupation','Rank', 'Probability', 'Label']]

print("oxfordDB Dimensions =>", oxfordDB.shape)
oxfordDB.head()

oxfordDB Dimensions => (702, 5)


,Code,Occupation,Rank,Probability,Label
0,29-1125,Recreational Therapists,1,0.0028,NaN
1,49-1011,"First-Line Supervisors of Mechanics, Installer...",2,0.0030,NaN
2,11-9161,Emergency Management Directors,3,0.0030,NaN
3,21-1023,Mental Health and Substance Abuse Social Workers,4,0.0031,NaN
4,29-1181,Audiologists,5,0.0033,NaN


## Data Collection

### 1. ONET + SOC + Oxford

In [28]:
datasets = [onetDB, socDB[socDB.columns.difference(['Occupation'])], oxfordDB[oxfordDB.columns.difference(['Occupation'])]]
pdMerge = lambda left,right: pd.merge(left,right,on='Code')
mergedDB = reduce(pdMerge, datasets)

print("onetDB{0} | socDB{1} | oxfordDB{2}".format(onetDB.shape,socDB.shape,oxfordDB.shape), end='\n\n')
print("mergedDB Dimensions =>",mergedDB.shape)
mergedDB.head()

onetDB(968, 11) | socDB(1106, 5) | oxfordDB(702, 5)

mergedDB Dimensions => (502, 17)


,Code,Occupation,SP,P,FD,O,AC,MD,CW,N,FA,Hourly Income,Industry Code,Total Emp,Label,Probability,Rank
0,19-3031,Counseling Psychologists,94,72,19,66,91,0,0,69,5,39.10,1235,108060,NaN,0.0047,24
1,19-3031,Clinical Psychologists,94,60,22,56,99,0,5,50,5,39.10,1235,108060,NaN,0.0047,24
2,19-3031,School Psychologists,75,63,22,50,62,0,6,60,7,39.10,1235,108060,NaN,0.0047,24
3,21-1013,Marriage and Family Therapists,81,72,38,66,94,0,1,69,5,25.89,1235,42880,0.0,0.0140,62
4,19-3039,Neuropsychologists and Clinical Neuropsycholog...,81,53,35,53,89,22,9,47,3,44.92,1235,12880,NaN,0.0043,17


### 2.1 ONET + SOC 

In [29]:
onetsocDB = pd.merge(onetDB,socDB[socDB.columns.difference(['Occupation'])] , on='Code')

print("onetDB{0} | socDB{1}".format(onetDB.shape,socDB.shape), end='\n\n')
print("onetsocDB Dimensions =>",onetsocDB.shape)
onetsocDB.head()

onetDB(968, 11) | socDB(1106, 5)

onetsocDB Dimensions => (608, 14)


,Code,Occupation,SP,P,FD,O,AC,MD,CW,N,FA,Hourly Income,Industry Code,Total Emp
0,19-3031,Counseling Psychologists,94,72,19,66,91,0,0,69,5,39.10,1235,108060
1,19-3031,Clinical Psychologists,94,60,22,56,99,0,5,50,5,39.10,1235,108060
2,19-3031,School Psychologists,75,63,22,50,62,0,6,60,7,39.10,1235,108060
3,29-1066,Psychiatrists,88,63,35,56,99,0,6,56,5,103.89,1235,25250
4,21-1013,Marriage and Family Therapists,81,72,38,66,94,0,1,69,5,25.89,1235,42880


### 2.2 ONET + Oxford

In [30]:
onetoxfordDB = pd.merge(onetDB,oxfordDB[oxfordDB.columns.difference(['Occupation'])] , on='Code')

print("onetDB{0} | oxfordDB{1}".format(onetDB.shape,oxfordDB.shape), end='\n\n')
print("onetoxfordDB Dimensions =>",onetoxfordDB.shape)
onetoxfordDB.head()

onetDB(968, 11) | oxfordDB(702, 5)

onetoxfordDB Dimensions => (844, 14)


,Code,Occupation,SP,P,FD,O,AC,MD,CW,N,FA,Label,Probability,Rank
0,19-3031,Counseling Psychologists,94,72,19,66,91,0,0,69,5,NaN,0.0047,24
1,19-3031,Clinical Psychologists,94,60,22,56,99,0,5,50,5,NaN,0.0047,24
2,19-3031,School Psychologists,75,63,22,50,62,0,6,60,7,NaN,0.0047,24
3,21-1014,Mental Health Counselors,94,66,28,60,94,0,3,56,15,NaN,0.0048,25
4,21-1013,Marriage and Family Therapists,81,72,38,66,94,0,1,69,5,0.0,0.0140,62


### 2.3 SOC + Oxford

In [31]:
socoxfordDB = pd.merge(socDB[socDB.columns.difference(['Occupation'])],oxfordDB[oxfordDB.columns.difference(['Occupation'])] , on='Code')

print("socDB{0} | oxfordDB{1}".format(socDB.shape,oxfordDB.shape), end='\n\n')
print("socoxfordDB Dimensions =>",socoxfordDB.shape)
socoxfordDB.head()

socDB(1106, 5) | oxfordDB(702, 5)

socoxfordDB Dimensions => (419, 7)


,Code,Hourly Income,Industry Code,Total Emp,Label,Probability,Rank
0,11-2021,70.01,1235,218970,NaN,0.0140,61
1,11-2022,66.18,1235,371410,NaN,0.0130,59
2,11-9031,25.74,1235,49130,0.0,0.0150,71
3,11-9032,NaN,1235,250280,NaN,0.0046,22
4,11-9033,51.77,1235,142160,NaN,0.0100,52


![](data/venn.png)